# Mixed heuristic approach

## Fast Simulated Annealing (FSA) & Genetic Optimization (GO)

### Author: Martin Zemko, HEUR, 2019

### Set Cover Problem
What is the set cover problem?
Idea:
“You must select a minimum number [of any size set] of these sets so that the sets you have picked contain all the elements that are contained in any of the sets in the input (wikipedia).” 
Additionally, you want to minimize the cost of the sets.

This task can be treated as a binary problem of linear programming, and its evaluation function is as follows:

$$f(x) = \sum_{i=1}^{m} x_i + \sum_{j=1}^{n} \lambda_i \cdot \mathrm{max} \left(1 - \sum_{k=1}^{m} S_{jk} \cdot x_k; 0 \right)$$

where

* m - set count
* n - point count

<img src="img/scp.png">

In [1]:
# Setting up enivironment
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

In [2]:
# Import external librarires
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
# Inicialization of evaluating functions
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)
def mne(x):
    return np.mean([n for n in x if n < np.inf])
def feo(x):
    return mne(x)/rel(x)
def go_boost(x):
    return np.sum(x) / len(x)

Initialization of the **Set Cover Problem** task with paramters as follows:
**``SCP(setCount=16, pointCount=16)``**

This task has 2^16 = 65 536 possible states

In [4]:
# Import our code
from objfun_scp import SCP
scp = SCP(setCount=16, pointCount=16)

## Random Shooting and Steepest Descent heuristic

In [5]:
from heur_sg import ShootAndGo

In [7]:
NUM_RUNS = 100
maxeval = 1000

In [8]:
def experiment_sg(of, maxeval, num_runs, hmax):
    results = []
    for i in tqdm_notebook(range(num_runs), 'Testing hmax={}'.format(hmax)):
        result = ShootAndGo(of, maxeval=maxeval, hmax=hmax).search() # dict with results of one run
        result['run'] = i
        result['heur'] = 'SG_{}'.format(hmax) # name of the heuristic
        result['hmax'] = hmax
        results.append(result)
    
    return pd.DataFrame(results, columns=['heur', 'run', 'hmax', 'best_x', 'best_y', 'neval'])

In [9]:
results_sg = pd.DataFrame()
for hmax in [0, np.inf]:
    res = experiment_sg(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, hmax=hmax)
    results_sg = pd.concat([results_sg, res], axis=0)

In [10]:
stats_sg = results_sg.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_sg = stats_sg.reset_index()
stats_sg

,heur,feo,mne,rel
0,SG_0,2121.440000,530.360000,0.25
1,SG_inf,255.024997,252.474747,0.99


## Fast Simulated Annealing heuristic

In [11]:
from heur_fsa import FastSimulatedAnnealing
from heur_aux import BinaryMutation, Correction

In [12]:
NUM_RUNS = 100
maxeval = 1000

In [13]:
def experiment_fsa(of, maxeval, num_runs, T0, n0, alpha, p):
    results = []
    for i in tqdm_notebook(range(num_runs), 'Testing T0={}, n0={}, alpha={}, p={}'.format(T0, n0, alpha, p)):
        mut = BinaryMutation(p=p, correction=Correction(of))
        result = FastSimulatedAnnealing(of, maxeval=maxeval, T0=T0, n0=n0, alpha=alpha, mutation=mut).search()
        result['run'] = i
        result['heur'] = 'FSA_{}_{}_{}_{}'.format(T0, n0, alpha, p) # name of the heuristic
        result['T0'] = T0
        result['n0'] = n0
        result['alpha'] = alpha
        result['p'] = p
        results.append(result)
    
    return pd.DataFrame(results, columns=['heur', 'run', 'T0', 'n0', 'alpha', 'p', 'best_x', 'best_y', 'neval'])

### Optimization of the initial temperature T0

In [14]:
results_fsa = pd.DataFrame()
for T0 in [1e-10, 1e-2, 1, np.inf]:
    res = experiment_fsa(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, T0=T0, n0=1, alpha=1, p=0.10)
    results_fsa = pd.concat([results_fsa, res], axis=0)

In [15]:
stats_fsa = results_fsa.pivot_table(
    index=['heur', 'T0'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa = stats_fsa.reset_index()
stats_fsa.sort_values(by=['T0'])
stats_fsa

,heur,T0,feo,mne,rel
0,FSA_0.01_1_1_0.1,1.000000e-02,118.340000,118.340000,1.00
1,FSA_1_1_1_0.1,1.000000e+00,117.050000,117.050000,1.00
2,FSA_1e-10_1_1_0.1,1.000000e-10,119.400000,119.400000,1.00
3,FSA_inf_1_1_0.1,inf,2740.720222,520.736842,0.19


### Optimization of the mutation probability p

In [16]:
results_fsa = pd.DataFrame()
for p in [0.01, 0.02, 0.05, 0.10, 0.20, 0.50]:
    res = experiment_fsa(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, T0=1, n0=1, alpha=1, p=p)
    results_fsa = pd.concat([results_fsa, res], axis=0)

In [17]:
stats_fsa = results_fsa.pivot_table(
    index=['heur', 'p'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa = stats_fsa.reset_index()
stats_fsa.sort_values(by=['p'])
stats_fsa

,heur,p,feo,mne,rel
0,FSA_1_1_1_0.01,0.01,411.045052,357.609195,0.87
1,FSA_1_1_1_0.02,0.02,250.124948,245.122449,0.98
2,FSA_1_1_1_0.05,0.05,140.590000,140.590000,1.00
3,FSA_1_1_1_0.1,0.10,129.950000,129.950000,1.00
4,FSA_1_1_1_0.2,0.20,145.080000,145.080000,1.00
5,FSA_1_1_1_0.5,0.50,1981.760000,495.440000,0.25


### Optimization of the cooling parameters n0 and alpha

In [20]:
results_fsa = pd.DataFrame()
for n0 in [1, 2, 5, 10, 100, np.inf]:
    res = experiment_fsa(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, T0=1, n0=n0, alpha=1, p=0.10)
    results_fsa = pd.concat([results_fsa, res], axis=0)

In [21]:
stats_fsa = results_fsa.pivot_table(
    index=['heur', 'n0'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa = stats_fsa.reset_index()
stats_fsa.sort_values(by=['n0'])
stats_fsa

,heur,n0,feo,mne,rel
0,FSA_1_100_1_0.1,100.0,211.970000,211.970000,1.00
1,FSA_1_10_1_0.1,10.0,131.430000,131.430000,1.00
2,FSA_1_1_1_0.1,1.0,113.130000,113.130000,1.00
3,FSA_1_2_1_0.1,2.0,113.650000,113.650000,1.00
4,FSA_1_5_1_0.1,5.0,114.190000,114.190000,1.00
5,FSA_1_inf_1_0.1,inf,413.926447,355.976744,0.86


In [26]:
results_fsa = pd.DataFrame()
for alpha in [1e-10, 1e-5, 1e-2, 1, 2, 5, 10]:
    res = experiment_fsa(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, T0=1, n0=1, alpha=alpha, p=0.10)
    results_fsa = pd.concat([results_fsa, res], axis=0)

In [27]:
stats_fsa = results_fsa.pivot_table(
    index=['heur', 'alpha'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa = stats_fsa.reset_index()
stats_fsa.sort_values(by=['alpha'])
stats_fsa

,heur,alpha,feo,mne,rel
0,FSA_1_1_0.01_0.1,1.000000e-02,226.665973,222.132653,0.98
1,FSA_1_1_10_0.1,1.000000e+01,97.820000,97.820000,1.00
2,FSA_1_1_1_0.1,1.000000e+00,99.630000,99.630000,1.00
3,FSA_1_1_1e-05_0.1,1.000000e-05,220.752984,218.545455,0.99
4,FSA_1_1_1e-10_0.1,1.000000e-10,226.343190,221.816327,0.98
5,FSA_1_1_2_0.1,2.000000e+00,121.760000,121.760000,1.00
6,FSA_1_1_5_0.1,5.000000e+00,108.100000,108.100000,1.00


The optimal FSA parameters are following:
* Initial temperature : T0 = 1
* Mutation probability: p = 0.10
* Cooling parameters  :
    - n0 = 1
    - alpha = 1

In [28]:
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
heur = FastSimulatedAnnealing(of=scp, maxeval=maxeval, T0=1, n0=1, alpha=1, mutation=mutation)
heur.reset()
result = heur.search()
print('neval = {}'.format(result['neval']))
print('best_x = {}'.format(result['best_x']))
print('best_y = {}'.format(result['best_y']))

neval = 31
best_x = [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
best_y = 3.0


## Genetic Optimization heuristic
Let's optimize the GO heuristic in order to use the results in the mixed heuristic

In [29]:
from heur_go import GeneticOptimization, UniformMultipoint
from heur_aux import BinaryMutation, Correction

In [30]:
NUM_RUNS = 100
maxeval = 1000

### Optimization of the size of the population

In [31]:
def experiment_go(of, maxeval, num_runs, N, M, Tsel1, Tsel2, mutation, crossover):
    results = []
    heur_name = 'GO_N={}'.format(N)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = GeneticOptimization(of, maxeval, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover).search()
        result['run'] = i
        result['heur'] = heur_name
        result['N'] = N
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'N', 'best_x', 'best_y', 'neval'])

In [32]:
results_go = pd.DataFrame()
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
crossover = UniformMultipoint(1)
for N in [1, 2, 3, 5, 10, 20, 30, 100]:
    res = experiment_go(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, N=N, M=N*3, Tsel1=1e-10, Tsel2=1e-2, 
                        mutation=mutation, crossover=crossover)
    results_go = pd.concat([results_go, res], axis=0)

In [33]:
stats_go = results_go.pivot_table(
    index=['heur', 'N'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_go = stats_go.reset_index()
stats_go.sort_values(by='N')
stats_go

,heur,N,feo,mne,rel
0,GO_N=1,1,214.723032,210.428571,0.98
1,GO_N=10,10,172.220000,172.220000,1.00
2,GO_N=100,100,327.335183,284.781609,0.87
3,GO_N=2,2,128.620000,128.620000,1.00
4,GO_N=20,20,215.463918,209.000000,0.97
5,GO_N=3,3,131.800000,131.800000,1.00
6,GO_N=30,30,265.329640,252.063158,0.95
7,GO_N=5,5,141.040000,141.040000,1.00


### Optimization of the mutation probability
Can we do better by changing the mutation probabilty?

In [34]:
def experiment_go_2(of, maxeval, num_runs, p,N, M, Tsel1, Tsel2, mutation, crossover):
    results = []
    heur_name = 'GO_p={}'.format(p)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = GeneticOptimization(of, maxeval, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover).search()
        result['run'] = i
        result['heur'] = heur_name
        result['p'] = p
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'p', 'best_x', 'best_y', 'neval'])

In [35]:
results_go_2 = pd.DataFrame()
crossover = UniformMultipoint(1)
N = 2
for p in [0.01, 0.02, 0.05, 0.10, 0.20, 0.50]:
    mutation = BinaryMutation(p=p, correction=Correction(scp))
    res = experiment_go_2(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, p=p, N=N, M=N*3, Tsel1=1e-10, Tsel2=1e-2, 
                        mutation=mutation, crossover=crossover)
    results_go_2 = pd.concat([results_go_2, res], axis=0)

In [36]:
stats_go_2 = results_go_2.pivot_table(
    index=['heur', 'p'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_go_2 = stats_go_2.reset_index()
stats_go_2.sort_values(by='p')
stats_go_2

,heur,p,feo,mne,rel
0,GO_p=0.01,0.01,467.209141,355.078947,0.76
1,GO_p=0.02,0.02,319.128072,293.597826,0.92
2,GO_p=0.05,0.05,136.410000,136.410000,1.00
3,GO_p=0.1,0.10,116.920000,116.920000,1.00
4,GO_p=0.2,0.20,262.886598,255.000000,0.97
5,GO_p=0.5,0.50,2228.117914,467.904762,0.21


### Optimization of the selection temperatures

In [39]:
def experiment_go_3(of, maxeval, num_runs, p,N, M, Tsel1, Tsel2, mutation, crossover):
    results = []
    heur_name = 'GO_Tsel1={}_Tsel2={}'.format(Tsel1, Tsel2)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = GeneticOptimization(of, maxeval, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover).search()
        result['run'] = i
        result['heur'] = heur_name
        result['Tsel1'] = Tsel1
        result['Tsel2'] = Tsel2
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'Tsel1', 'Tsel2', 'best_x', 'best_y', 'neval'])

In [43]:
results_go_3 = pd.DataFrame()
crossover = UniformMultipoint(1)
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
N = 2
for Tsel in [1e-10, 1e-2, 1, np.inf]:
    res = experiment_go_3(of=scp, maxeval=maxeval, num_runs=NUM_RUNS, p=p, N=N, M=N*3, Tsel1=1e-10, Tsel2=Tsel, 
                        mutation=mutation, crossover=crossover)
    results_go_3 = pd.concat([results_go_3, res], axis=0)

In [44]:
stats_go_3 = results_go_3.pivot_table(
    index=['heur', 'Tsel2'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_go_3 = stats_go_3.reset_index()
stats_go_3.sort_values(by='Tsel2')
stats_go_3

,heur,Tsel2,feo,mne,rel
0,GO_Tsel1=1e-10_Tsel2=0.01,1.000000e-02,122.25,122.25,1.0
1,GO_Tsel1=1e-10_Tsel2=1,1.000000e+00,136.66,136.66,1.0
2,GO_Tsel1=1e-10_Tsel2=1e-10,1.000000e-10,134.90,134.90,1.0
3,GO_Tsel1=1e-10_Tsel2=inf,inf,NaN,NaN,0.0


The optimal GO parameters are following:
* Size of the population: N = 2
* Mutation probability  : p = 0.10
* Selection temperatures:
    - Tsel1 = 1e-10
    - Tsel2 = 1e-2

In [45]:
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
heur = GeneticOptimization(of=scp, maxeval=maxeval, N=2, M=6, Tsel1=1e-10, Tsel2=1e-2, 
                        mutation=mutation, crossover=crossover)
result = heur.search()
print('neval = {}'.format(result['neval']))
print('best_x = {}'.format(result['best_x']))
print('best_y = {}'.format(result['best_y']))

neval = 208
best_x = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
best_y = 3.0


## Fast Simulated Annealing + Genetic Optimization heursitic
In this case, we will use FSA and GO parameters optimized separately in the previous section.

In [46]:
from heur_go import UniformMultipoint
from heur_fsa_go import FsaGoHeuristic
from heur_aux import BinaryMutation, Correction

### Optimization of the size of the population
We will find out whether the parameters from the GO heuristic are optimal in the combined heuristic as well

In [47]:
NUM_RUNS = 100
maxevalFsa = 50
maxevalGo = 1000

In [48]:
def experiment_fsa_go(of, maxevalFsa, maxevalGo, num_runs, N, M, Tsel1, Tsel2, mutation, crossover, T0, n0, alpha):
    results = []
    heur_name = 'FSA_GO_N={}'.format(N)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = FsaGoHeuristic(of=of, maxevalFsa=maxevalFsa, maxevalGo=maxevalGo, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover,
                                        T0=T0, n0=n0, alpha=alpha).search()
        result['run'] = i
        result['heur'] = heur_name
        result['N'] = N
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'N', 'best_x', 'best_y', 'neval', 'GO_boost'])

In [49]:
results_fsa_go_1 = pd.DataFrame()
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
crossover = UniformMultipoint(1)
for N in [1, 2, 3, 5, 10, 20, 50]:
    res = experiment_fsa_go(of=scp, maxevalFsa=maxevalFsa, maxevalGo=maxevalGo, num_runs=NUM_RUNS, N=N, M=N*3, Tsel1=1e-10, Tsel2=1e-2, 
                        mutation=mutation, crossover=crossover, T0=1, n0=1, alpha=1)
    results_fsa_go_1 = pd.concat([results_fsa_go_1, res], axis=0)

In [50]:
stats_fsa_go_1A = results_fsa_go_1.pivot_table(
    index=['heur', 'N'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa_go_1A = stats_fsa_go_1A.reset_index()
stats_fsa_go_1A.sort_values(by='N')
stats_fsa_go_1A

,heur,N,feo,mne,rel
0,FSA_GO_N=1,1,187.225793,185.353535,0.99
1,FSA_GO_N=10,10,170.280000,170.280000,1.00
2,FSA_GO_N=2,2,159.780000,159.780000,1.00
3,FSA_GO_N=20,20,178.580000,178.580000,1.00
4,FSA_GO_N=3,3,169.320000,169.320000,1.00
5,FSA_GO_N=5,5,187.330000,187.330000,1.00
6,FSA_GO_N=50,50,182.960000,182.960000,1.00


In [51]:
stats_fsa_go_1B = results_fsa_go_1.pivot_table(
    index=['heur', 'N'],
    values=['GO_boost'],
    aggfunc=(go_boost)
)['GO_boost']
stats_fsa_go_1B = stats_fsa_go_1B.reset_index()
stats_fsa_go_1B.sort_values(by='N')
stats_fsa_go_1B

,heur,N,GO_boost
0,FSA_GO_N=1,1,0.70
1,FSA_GO_N=10,10,0.05
2,FSA_GO_N=2,2,0.60
3,FSA_GO_N=20,20,0.00
4,FSA_GO_N=3,3,0.41
5,FSA_GO_N=5,5,0.26
6,FSA_GO_N=50,50,0.00


The best combined heuristisic has only 2 elements in the population. Let's tune other parameters!

### Optimization of the FSA cooling factor

In [52]:
def experiment_fsa_go_2(of, maxevalFsa, maxevalGo, num_runs, N, M, Tsel1, Tsel2, mutation, crossover, T0, n0, alpha):
    results = []
    heur_name = 'FSA_GO_n0={}'.format(n0)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = FsaGoHeuristic(of=of, maxevalFsa=maxevalFsa, maxevalGo=maxevalGo, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover,
                                        T0=T0, n0=n0, alpha=alpha).search()
        result['run'] = i
        result['heur'] = heur_name
        result['n0'] = n0
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'n0', 'best_x', 'best_y', 'neval', 'GO_boost'])

In [53]:
results_fsa_go_2 = pd.DataFrame()
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
crossover = UniformMultipoint(1)
for n0 in [1, 2, 5, 10]:
    res = experiment_fsa_go_2(of=scp, maxevalFsa=50, maxevalGo=maxevalGo, num_runs=NUM_RUNS, N=2, M=6, Tsel1=1e-10, Tsel2=1e-2,
                              mutation=mutation, crossover=crossover, T0=1, n0=n0, alpha=1)
    results_fsa_go_2 = pd.concat([results_fsa_go_2, res], axis=0)

In [54]:
stats_fsa_go_2A = results_fsa_go_2.pivot_table(
    index=['heur', 'n0'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa_go_2A = stats_fsa_go_2A.reset_index()
stats_fsa_go_2A.sort_values(by='n0')
stats_fsa_go_2A

,heur,n0,feo,mne,rel
0,FSA_GO_n0=1,1,133.42,133.42,1.0
1,FSA_GO_n0=10,10,188.89,188.89,1.0
2,FSA_GO_n0=2,2,167.81,167.81,1.0
3,FSA_GO_n0=5,5,147.33,147.33,1.0


In [55]:
stats_fsa_go_2B = results_fsa_go_2.pivot_table(
    index=['heur', 'n0'],
    values=['GO_boost'],
    aggfunc=(go_boost)
)['GO_boost']
stats_fsa_go_2B = stats_fsa_go_2B.reset_index()
stats_fsa_go_2B.sort_values(by='n0')
stats_fsa_go_2B

,heur,n0,GO_boost
0,FSA_GO_n0=1,1,0.50
1,FSA_GO_n0=10,10,0.70
2,FSA_GO_n0=2,2,0.57
3,FSA_GO_n0=5,5,0.62


Optimization of the mixed heuristic provides the same results as optimization of the separated FSA and GO heuristics.

### Finding the FSA cut off

In [56]:
def experiment_fsa_go_3(of, maxevalFsa, maxevalGo, num_runs, N, M, Tsel1, Tsel2, mutation, crossover, T0, n0, alpha):
    results = []
    heur_name = 'FSA_GO_{}'.format(maxevalFsa)
    for i in tqdm_notebook(range(num_runs), 'Testing {}'.format(heur_name)):
        result = FsaGoHeuristic(of=of, maxevalFsa=maxevalFsa, maxevalGo=maxevalGo, N=N, M=M, Tsel1=Tsel1, Tsel2=Tsel2, 
                                     mutation=mutation, crossover=crossover,
                                        T0=T0, n0=n0, alpha=alpha).search()
        result['run'] = i
        result['heur'] = heur_name
        result['maxevalFsa'] = maxevalFsa
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'maxevalFsa', 'best_x', 'best_y', 'neval', 'GO_boost'])

In [57]:
results_fsa_go_3 = pd.DataFrame()
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
crossover = UniformMultipoint(1)
for maxevalFsa in [2, 5, 10, 20, 30, 50, 100, 200, 500]:
    res = experiment_fsa_go_3(of=scp, maxevalFsa=maxevalFsa, maxevalGo=maxevalGo, num_runs=NUM_RUNS, N=2, M=N*3, Tsel1=1e-10, Tsel2=1e-2, 
                        mutation=mutation, crossover=crossover, T0=1, n0=1, alpha=1)
    results_fsa_go_3 = pd.concat([results_fsa_go_3, res], axis=0)

In [58]:
stats_fsa_go_3A = results_fsa_go_3.pivot_table(
    index=['heur', 'maxevalFsa'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_fsa_go_3A = stats_fsa_go_3A.reset_index()
stats_fsa_go_3A.sort_values(by='maxevalFsa')
stats_fsa_go_3A

,heur,maxevalFsa,feo,mne,rel
0,FSA_GO_10,10,461.721585,383.228916,0.83
1,FSA_GO_100,100,175.281133,171.775510,0.98
2,FSA_GO_2,2,447.493827,402.744444,0.90
3,FSA_GO_20,20,417.674858,384.260870,0.92
4,FSA_GO_200,200,130.060000,130.060000,1.00
5,FSA_GO_30,30,366.506173,329.855556,0.90
6,FSA_GO_5,5,417.169549,371.280899,0.89
7,FSA_GO_50,50,243.379501,231.210526,0.95
8,FSA_GO_500,500,106.630000,106.630000,1.00


In [59]:
stats_fsa_go_3B = results_fsa_go_3.pivot_table(
    index=['heur', 'maxevalFsa'],
    values=['GO_boost'],
    aggfunc=(go_boost)
)['GO_boost']
stats_fsa_go_3B = stats_fsa_go_3B.reset_index()
stats_fsa_go_3B.sort_values(by='maxevalFsa')
stats_fsa_go_3B

,heur,maxevalFsa,GO_boost
0,FSA_GO_10,10,1.00
1,FSA_GO_100,100,0.20
2,FSA_GO_2,2,0.97
3,FSA_GO_20,20,0.93
4,FSA_GO_200,200,0.02
5,FSA_GO_30,30,0.74
6,FSA_GO_5,5,1.00
7,FSA_GO_50,50,0.45
8,FSA_GO_500,500,0.00


With respect to the result, we should cut off the FSA initialization after 500 evaluations (or maybe more). However, it means that we should completely eliminate the Geneteric Optimization and let the FSA heuristic calculate the solution.

The optimal parameters of the mixed heuristic are following:
* FSA max evaluations    : >500
* Initial temperature    : T0 = 1
* Mutation probability   : p = 0.10
* Size of the population : N = 2
* Cooling parameters :
    - n0 = 1
    - alpha = 1
* Selection temperatures :
    - Tsel1 = 1e-10
    - Tsel2 = 1e-2

In [60]:
mutation = BinaryMutation(p=0.10, correction=Correction(scp))
heur = FsaGoHeuristic(of=scp, maxevalFsa=500, maxevalGo=maxevalGo, N=2, M=6, Tsel1=10e-10, Tsel2=1e-2, 
                                     mutation=mutation, crossover=crossover,
                                        T0=1, n0=1, alpha=1)
result = heur.search()
print('neval = {}'.format(result['neval']))
print('best_x = {}'.format(result['best_x']))
print('best_y = {}'.format(result['best_y']))

neval = 63
best_x = [1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
best_y = 3.0


## Results

Shoot and Go result:

In [61]:
stats_sg

,heur,feo,mne,rel
0,SG_0,2121.440000,530.360000,0.25
1,SG_inf,255.024997,252.474747,0.99


Fast Simulated Annealing heuristic result:

In [62]:
stats_fsa.sort_values(by=['feo']).head(1)

,heur,alpha,feo,mne,rel
1,FSA_1_1_10_0.1,10.0,97.82,97.82,1.0


Genetic Optimization heuristic result:

In [63]:
stats_go_2.sort_values(by=['feo']).head(1)

,heur,p,feo,mne,rel
3,GO_p=0.1,0.1,116.92,116.92,1.0


Mixed (GO + FSA) heuristic result:

In [64]:
stats_fsa_go_3A.sort_values(by=['feo']).head(1)

,heur,maxevalFsa,feo,mne,rel
8,FSA_GO_500,500,106.63,106.63,1.0


## Conclusion

1. Based on the FEO values, the best heuristics solving the SCP problem is the FSA heuristics.
2. The mixed heuristics is the second best heuristics because it involves the FSA in the first phase. Moreover, its optimization process eliminates the second phase (i.e. the GO heuristics). Thus, the result is calculated only by the FSA.
3. Using the mixed approach, we can achieve better results than in the RandomShooting heuristics. However, it is because of the built-in FSA taking control over the problem.
4. The mixed approach is definitely not better than FSA in this particular case.
5. The mixed heuristics can be useful only for making decision about which heuristics should be used (FSA or GO). Besides that, this heuristics does not provide any additional benefits.